In [12]:
import numpy as np
import pandas as pd
import string, xlrd, nltk, unicodedata, pickle, os, re, time
import xml.etree.cElementTree as ET
from collections import defaultdict, Counter

start = time.time()

# loads the spreadsheet as a pandas.DataFrame
xls_file = pd.ExcelFile("Nykredit_feed.xls")
print (xls_file.sheet_names)
nykredit_data = xls_file.parse('tmp62.tmp')
nykredit_text = xls_file.parse('ArticleText')

id_XML = defaultdict(str)

def xml_to_text(row):
    """
    Parses the raw XML string in row.
    """
    try: # not all XML strings are well formed
        tree = ET.ElementTree(ET.fromstring(row['Raw Xml']))
    except:
        return None
    
    paragraphs = []
    for elem in tree.iter('p'): #  The relevant text is enclosed in multiple p tags.
        paragraphs.append(elem.text)
    paragraphs = [p for p in paragraphs if p!=None] # removes empty paragraphs
    text = " ".join(paragraphs)
    
    return text

selected_parameters = ['ArticleKey', 'ArticleDate', 'QualitativeScore', 'Headline', 'Kilde', 'Raw Xml']
selected_data = nykredit_data.reindex(columns=selected_parameters)

selected_data['text'] = selected_data.apply(xml_to_text, axis=1)

selected_data

['tmp62.tmp', 'ArticleText']


,ArticleKey,ArticleDate,QualitativeScore,Headline,Kilde,Raw Xml,text
0,e25f3cf8,2010-12-28,0,Fortsat lav rente på flexlån,Hadsund Folkeblad,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",ALS: December måned ventes hvert år med spændi...
1,e25bea65,2010-12-21,0,Fortsat en meget lav rente på et-årige flexlån,Aars Avis,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",AARS: December ventes hvert år med spænding af...
2,e252432b,2010-12-01,1,Førstegangskøbere kan med fordel købe bolig nu,Aabenraa Ugeavis,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",BOLIG: » Mange førstegangskøbere kan med forde...
3,e23bc88a,2010-10-05,1,Sparbank Nord anbefaler at lægge huslånene om,Annonce Bladet Salling-Fur-Skive,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",I de seneste måneder har massevis af boligejer...
4,e2520d69,2010-12-01,1,Nu kan der med fordel købes bolig,Lokal-Bladet Budstikken Vejen,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",-Mange førstegangskøbere kan med fordel slå ti...
5,e2557926,2010-12-08,1,Førstegangskøber af hus?,Lokal-Bladet Budstikken Tønder,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",Mange førstegangskøbere kan til egen fordel sl...
6,e255d917,2010-12-09,1,Fyrer du for gråspurvene?,Lokalavisen Vanløse Posten,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",Lader du store besparelser på elregningen slip...
7,e2534aee,2010-12-01,1,Fyrer du for gråspurvene?,Lokalavisen 2500 Valby 2450 Sydhavnen,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...",Lader du store besparelser på elregningen slip...
8,e2552647,2010-12-08,1,En varm hilsen fra realkreditinstitut,Randers Onsdag,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...","RENTETILPASNINGSLÅN: Inden december er omme, k..."
9,e25e2b7d,2010-12-26,0,Sådan ser renten ud i 2011,"Søndagsavisen 10C (Præstø, Vordingborg, Næstved)","<?xml version=""1.0"" encoding=""utf-8""?><NewsML ...","»Det afspejler, at selskaberne er kommet forsk..."


In [39]:
selected_data['BagOfWords'] = selected_data.text

In [83]:
strings = ["asdf sdaf asd fasd fasdf j", "asdf ljfi eoi joi erog ognais dnseafn dfnls dfn 2f"]
test = pd.DataFrame(strings)
test.columns = ["firstColumn"]

In [87]:
test["splits"] = 1
test.splits =  test.splits * 10
test

,firstColumn,splits
0,asdf sdaf asd fasd fasdf j,10
1,asdf ljfi eoi joi erog ognais dnseafn dfnls df...,10


In [6]:
nykredit_data
id_list = []
for article in nykredit_data:
    id_list.append(article)

In [7]:
tree = ET.ElementTree(ET.fromstring(nykredit_data['Raw Xml'][0]))
#tree = ElementTree.fromstring(nykredit_data['Raw Xml'][0])
content = tree.findall('NewsItem/NewsComponent/NewsComponent/ContentItem/DataContent/nitf/body/body.content/block/p')

headline = tree.findall('NewsItem')
print (headline)

[<Element 'NewsItem' at 0x115ea4318>]


In [88]:
nykredit_data = xls_file.parse('tmp62.tmp')
nykredit_text = xls_file.parse('ArticleText')

In [89]:
nykredit_data

,QueryID,ObjectName,ArticleKey,ArticleDate,ArticleYear,ArticleMonth,ArticleWeek,ArticleDay,SourceID,Kilde,...,XField07,XField08,XField09,XField10,Country,WorldRegion,Kommentar,PRFactor,PREffect,Raw Xml
0,2400,Totalkredit,e25f3cf8,2010-12-28,2010,12,52,28,HAF,Hadsund Folkeblad,...,NaN,NaN,NaN,NaN,Danmark,Norden,Ny fleksrente,2.0,420.000,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
1,2400,Totalkredit,e25bea65,2010-12-21,2010,12,51,21,AAR,Aars Avis,...,NaN,NaN,NaN,NaN,Danmark,Norden,Ny fleksrente,2.0,580.000,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
2,2400,Totalkredit,e252432b,2010-12-01,2010,12,48,1,AAU,Aabenraa Ugeavis,...,NaN,NaN,NaN,NaN,Danmark,Norden,NaN,30.0,4500.000,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
3,2400,Totalkredit,e23bc88a,2010-10-05,2010,10,40,5,ABS,Annonce Bladet Salling-Fur-Skive,...,NaN,NaN,NaN,NaN,Danmark,Norden,NaN,33.5,6700.000,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
4,2400,Totalkredit,e2520d69,2010-12-01,2010,12,48,1,BUN,Lokal-Bladet Budstikken Vejen,...,NaN,NaN,NaN,NaN,Danmark,Norden,NaN,30.0,8132.400,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
5,2400,Totalkredit,e2557926,2010-12-08,2010,12,49,8,BUO,Lokal-Bladet Budstikken Tønder,...,NaN,NaN,NaN,NaN,Danmark,Norden,NaN,30.0,6042.300,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
6,2400,Totalkredit,e255d917,2010-12-09,2010,12,49,9,LVP,Lokalavisen Vanløse Posten,...,NaN,NaN,NaN,NaN,Danmark,Norden,Energi,32.0,11000.000,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
7,2400,Totalkredit,e2534aee,2010-12-01,2010,12,48,1,LVS,Lokalavisen 2500 Valby 2450 Sydhavnen,...,NaN,NaN,NaN,NaN,Danmark,Norden,Energi,4.0,2168.400,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
8,2400,Totalkredit,e2552647,2010-12-08,2010,12,49,8,RAO,Randers Onsdag,...,NaN,NaN,NaN,NaN,Danmark,Norden,Ny fleksrente,30.0,16800.000,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
9,2400,Totalkredit,e25e2b7d,2010-12-26,2010,12,51,26,S0C,"Søndagsavisen 10C (Præstø, Vordingborg, Næstved)",...,NaN,NaN,NaN,NaN,Danmark,Norden,Ny fleksrente,9.0,9450.000,"<?xml version=""1.0"" encoding=""utf-8""?><NewsML ..."
